In [1]:

import cv2 as cv
import numpy as np
import imutils

### Open CV

In [2]:
def show_img(text,src):
    cv.imshow(text,src)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv.resize(imgArray[x][y], (imgArray[0][0].shape[1],
                                                                imgArray[0][0].shape[0]),None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv.cvtColor( imgArray[x][y], cv.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv.cvtColor(imgArray[x], cv.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

#### Resize and Rotate

In [5]:
img=cv.imread('my_image/me.jpg')
img_resized=cv.resize(img,(300,200))
show_img('Fixed Resizing',img_resized)

In [ ]:
img_resized_aspect_ratio=imutils.resize(img,width=300)
show_img('Aspect Ratio Resize',img_resized_aspect_ratio)

In [ ]:
img_rotated_pos=imutils.rotate(img,45)
img_rotated_neg=imutils.rotate(img,-45)
img_rotated = stackImages(0.5,([img,img_rotated_pos],[img,img_rotated_neg]))

show_img('Rotation',img_rotated)

In [ ]:
img_rotated_bound=imutils.rotate_bound(img,45)
show_img('Rotation Bound',img_rotated_bound)

#### Counting Objects

In [ ]:

img_shapes=cv.imread('chapter04-opencv_tutorial\images\shapes.png')
img_copy=img_shapes.copy()
img_gray=cv.cvtColor(img_shapes,cv.COLOR_BGR2GRAY)
img_blur=cv.GaussianBlur(img_gray,(3,3),0)
img_edged=cv.Canny(img_blur,50,150)


cnts=cv.findContours(img_edged,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
# list of contours 
cnts=imutils.grab_contours(cnts)

total=0
for cnt in cnts:
    # ignore small areas (noises)
    if cv.contourArea(cnt) <25:
        continue
    cv.drawContours(img_copy,[cnt],-1,(204,0,255),2)
    total+=1
    
    peri=cv.arcLength(cnt,True) #curve length 
    # print(peri)
    approx=cv.approxPolyDP(cnt,epsilon=0.02*peri,closed=True)
    print(len(approx)) #3=> triangle #4 =>rectangle or Square #4> => curve

    obj_cor=len(approx)
    
    x,y,w,h=cv.boundingRect(approx)
    
    obj_type=''
    if obj_cor==3:
        obj_type='Triangle'
    elif obj_cor==5:
        obj_type='Pentagon'
    elif obj_cor==6:
        obj_type='WTFFF'
    elif obj_cor==4:
        asp_ratio=w/float(h)
        if asp_ratio >0.95 and asp_ratio < 1.05 :
            obj_type='Square'
        else:
            obj_type='Rectangle'
    else:
        obj_type='Curve'
    
    cv.rectangle(img_copy,(x,y),(x+w,y+h),(0,255,0),2)
    cv.putText(img_copy,obj_type,
                (x+(w//2)-20,y+(h//2)),cv.FONT_HERSHEY_COMPLEX,0.6,
                (56, 32, 24),2)   
    
print(f'**** {total} objects found **** ')

img_stack = stackImages(0.5,([img_shapes,img_gray,img_copy],[img_blur,img_edged,img_copy]))
show_img('Counting Objects',img_stack)

5
4
6
4
8
3
4
4
8
4
8
4
4
5
4
4
5
**** 17 objects found **** 


#### Image Subtraction

**vars :** The method returns the __dict__ attribute for a module, class, instance, or any other object if the same has a __dict__ attribute. If the object fails to match the attribute, it raises a TypeError exception. Objects such as modules and instances have an updatable __dict__ attribute however, other objects may have written restrictions on their __dict__ attributes. vars() acts like locals() method when an empty argument is passed which implies that the locals dictionary is only useful for reads since updates to the locals dictionary are ignored. 

In [ ]:


# import argparse

# # construct the argument parser and parse the arguments

# ap=argparse.ArgumentParser()

# ap.add_argument('-b','--bg', required=True,help='path to the background image')
# ap.add_argument('-f','--fg', required=True,help='path to the foreground image')

# args=vars(ap.parse_args())

# img_bg=cv.imread(args['bg'])
# img_fg=cv.imread(args['fg'])

# # execute this script 

# python [NAME].py --bg [BG_PATH] --fg [FG_PATH]  




In [ ]:
# load images
img_bg=cv.imread("my_image\me_bg.jpg")
img_fg=cv.imread("my_image\me_fg.jpg")

In [ ]:

# show images
# show_img('img_bg',img_bg)
# show_img('img_fg',img_fg)

# convert background and foreground images to grayscale
img_bg_gray=cv.cvtColor(img_bg,cv.COLOR_BGR2GRAY)
img_fg_gray=cv.cvtColor(img_fg,cv.COLOR_BGR2GRAY)

# perform background subtraction | background image (int32) - foreground image (int32) |
img_sub=img_bg_gray.astype('int32')-img_fg_gray.astype('int32')
img_sub=np.absolute(img_sub).astype('uint8')

print(f'bg : {img_bg_gray[0][:13]} ,\nfg : {img_fg_gray[0][:13]} ,\nsub : {img_sub[0][:13]} ')

show_img('img_sub',img_sub)

bg : [71 72 73 74 74 74 75 76 75 75 74 73 73] ,
fg : [43 43 42 43 44 46 47 47 48 47 48 49 48] ,
sub : [28 29 31 31 30 28 28 29 27 28 26 24 25] 


**Basics of Erosion:** 
 
Erodes away the boundaries of the foreground object <br>
Used to diminish the features of an image. <br> <br>


**Basics of dilation:**
 

Increases the object area <br>
Used to accentuate features

In [ ]:
img_thresh=cv.threshold(img_sub,0,255,cv.THRESH_BINARY | cv.THRESH_OTSU)[1]
img_erode=cv.erode(img_thresh,None,iterations=1)
img_dilate=cv.dilate(img_erode,None,iterations=1)

# # show images
# img_subtraction = stackImages(0.7,([img_bg_gray,img_fg_gray,img_sub],[img_thresh,img_erode,img_dilate]))
# show_img('img_subtraction',img_subtraction)

contour detection

In [ ]:
# find contours in the thresholded difference map and then initialize our bounding box regions
# that contains the *entire* region of motion. 

cnts=cv.findContours(img_thresh.copy(),cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
cnts=imutils.grab_contours(cnts)

(min_x,min_y)=(np.inf,np.inf)
(max_x,max_y)=(-np.inf,-np.inf)

# loop over the contours
for c in cnts:
    # compute the bounding box of the contour
    (x,y,w,h)=cv.boundingRect(c)
    
    
    # reduse noises by enforcing  requirements on the bounding box size 
    if w>20 and h>20:
        # update our bookkeeping variables
        min_x=min(min_x,x)
        min_y=min(min_y,y)
        max_x=max(max_x,x+w-1)
        max_y=max(max_y,y+h-1)
        
# draw a rectangle surrounding the region of motion
cv.rectangle(img_fg,(min_x,min_y), (max_x,max_y),(0,255,0) ,3)

# show images
img_subtraction = stackImages(0.3,([img_bg_gray,img_fg_gray,img_sub],[img_thresh,img_dilate,img_fg]))
show_img('img_subtraction',img_subtraction)

##### Object and Face Detection

In [ ]:

# load images
img_family=cv.imread('my_image\\family.jpg')

# resize image
img_family=imutils.resize(img_family,width=700)

# convert image to grayscale
img_family_gray=cv.cvtColor(img_family,cv.COLOR_BGR2GRAY)


**detectMultiScale :**

**scaleFactor** – Parameter specifying how much the image size is reduced at each image scale.<br>
1.05 is a good possible value for this, which means you use a small step for resizing, i.e. reduce the size by 5%, you increase the chance of a matching size with the model for detection is found.<br><br>


**minNeighbors** – Parameter specifying how many neighbors each candidate rectangle should have to retain it. <br>
This parameter will affect the quality of the detected faces. Higher value results in fewer detections but with higher quality. 3~6 is a good value for it.<br><br>

**minSize** – Minimum possible object size. Objects smaller than that are ignored.<br>
This parameter determines how small size you want to detect. You decide it! Usually, [30, 30] is a good start for face detection.<br><br>

**maxSize** – Maximum possible object size. Objects bigger than this are ignored.
This parameter determines how big size you want to detect. Again, you decide it! Usually, you don't need to set it manually, the default value assumes you want to detect without an upper limit on the size of the face.<br><br>

In [ ]:
# load face detector 
detector=cv.CascadeClassifier('haarcascade_frontalface_alt_tree.xml')

# detect faces in the image
rects=detector.detectMultiScale(img_family_gray,scaleFactor=1.05,minNeighbors=3,minSize=(20,20),flags=cv.CASCADE_SCALE_IMAGE)

# total number of faces in image
print(f'[INFO] detected {len(rects)}')

[INFO] detected 6


In [ ]:
# loop over the bounding boxes and draw a rectangle around each face
for (x,y,w,h) in rects:

    cv.rectangle(img_family,(x,y), (x+w,y+h),(0,255,0) ,2)

show_img('img family',img_family)

#### Access Camera

In [4]:
from imutils.video import VideoStream
import time 

In [5]:

print('[INFO] starting video stream ')
# initialize the video stream
vs = VideoStream(src=-1).start()
# vs=VideoStream(src=0,usePiCamera=True,resolution=(640,480))

# start thewebcam video stream and turn off the autofoucos setting
vs.stream.set(cv.CAP_PROP_AUTOFOCUS,0)

time.sleep(2.0)

# loop over the frames from the video stream 
while True:
    # grab the frame from the video stream and resize it
    frame= vs.read()
    frame = imutils.resize(frame,width=400)
    
    # show the output frame
    cv.imshow('Frame',frame)
    key= cv.waitKey(1) & 0xFF
    
    # if the 'q' ker was pressed, break from the loop
    if key== ord('q'):
        break

# do a bit for cleanup
cv.destroyAllWindows()
vs.stop()

[INFO] starting video stream 


In [20]:
def toggle_autofocus(vs,autofocus=True):
    # set the autofocus camera property  ON or OFF
    vs.stream.set(cv.CAP_PROP_AUTOFOCUS,1 if autofocus else 0)
    print(f'[INFO] autofocus has been set to {"ON" if autofocus else "OFF"}')

    # read back the property to ensure it was set
    actualAutofocus=vs.stream.get(cv.CAP_PROP_AUTOFOCUS)
    print(f'[INFO] actual autofocus {actualAutofocus}')


def toggle_auto_whitebalance(vs,autowb=True):
    # set the auto whitebalance camera property  ON or OFF
    vs.stream.set(cv.CAP_PROP_AUTO_WB,1 if autowb else 0)
    print(f'[INFO] auto white balance has been set to {"ON" if autowb else "OFF"}')

    # read back the property to ensure it was set
    actualAutoWB=vs.stream.get(cv.CAP_PROP_AUTO_WB)
    print(f'[INFO] actual auto white balance {actualAutoWB}')

def set_zoom(vs,zoom=100):
    # set the zoom camera property  ON or OFF
    vs.stream.set(cv.CAP_PROP_ZOOM,zoom)
    print(f'[INFO] zoom has been set to {zoom}')

    # read back the property to ensure it was set
    actualZoom=vs.stream.get(cv.CAP_PROP_ZOOM)
    print(f'[INFO] actual zoom {actualZoom}')



[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 7
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[http @ 0x10a3d40] Stream ends prematurely at 1219647591, should be 18446744073709551615


In [19]:

print('[INFO] starting video stream ')
# initialize the video stream
url='http://192.168.0.190:8080/video'
vs = VideoStream(src=url).start()

# initialize the camera parameter settings
autofocus=True
autowb=True
zoom=100
# vs.stream
# vs.set(cv.CAP_PROP_ZOOM,50)
# time.sleep(2.0)

# loop over the frames from the video stream 
while True:
    # grab the frame from the video stream and resize it
    frame= vs.read()
    frame = imutils.resize(frame,width=600)
    
    # show the output frame
    cv.imshow('Frame',frame)
    key= cv.waitKey(1)
    
    # handle *q* keypresses for "quit"

    if key== ord('q'):
        break  

    # handle *f* keypresses for "autofocus"
    elif key== ord('f'):
        # toggle autofocus and set the camera property
        autofocus= not autofocus
        toggle_autofocus(vs,autofocus)

    # handle *w* keypresses for "auto white balance"
    elif key== ord('w'):
        # toggle auto white balance and set the camera property
        autowb= not autowb
        toggle_auto_whitebalance(vs,autowb)

    # handle *i* keypresses for "zoom in"
    elif key== ord('i'):
        # increase zoom  and set the camera property
        zoom+=1
        set_zoom(vs,zoom)

    # handle *o* keypresses for "zoom out"
    elif key== ord('o'):
        # decrease zoom  and set the camera property
        zoom-=1
        set_zoom(vs,zoom)

# reset camera parameter settings
toggle_autofocus(vs,True)
toggle_auto_whitebalance(vs,True)
set_zoom(vs,100)

# do a bit for cleanup
vs.stop()
cv.destroyAllWindows()

[INFO] starting video stream 
[INFO] auto white balance has been set to OFF
[INFO] actual auto white balance 0.0
[INFO] auto white balance has been set to ON
[INFO] actual auto white balance 0.0
[INFO] auto white balance has been set to OFF
[INFO] actual auto white balance 0.0
[INFO] auto white balance has been set to ON
[INFO] actual auto white balance 0.0
[INFO] auto white balance has been set to OFF
[INFO] actual auto white balance 0.0
[INFO] auto white balance has been set to ON
[INFO] actual auto white balance 0.0
[INFO] zoom has been set to 101
[INFO] actual zoom 0.0
[INFO] zoom has been set to 102
[INFO] actual zoom 0.0
[INFO] zoom has been set to 103
[INFO] actual zoom 0.0
[INFO] zoom has been set to 102
[INFO] actual zoom 0.0
[INFO] zoom has been set to 101
[INFO] actual zoom 0.0
[INFO] zoom has been set to 100
[INFO] actual zoom 0.0
[INFO] zoom has been set to 99
[INFO] actual zoom 0.0
[INFO] auto white balance has been set to OFF
[INFO] actual auto white balance 0.0
[INFO] z

[mjpeg @ 0x1ae9900] overread 3
[mjpeg @ 0x1ae9900] overread 3
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8
[mjpeg @ 0x1ae9900] overread 8


#### Time Lapse

##### Capture Time Lapse Frames 

In [2]:
from imutils.video import VideoStream
from datetime import datetime
import argparse
import signal
import time
import sys
import os

In [3]:
def signal_handler(sig,frame):
    print('[INFO] You pressed `ctrl + c`! Your pictures are saved' \
          ' in the output directory you specified :) ')
    sys.exit(0)


# ap=argparse.ArgumentParser()

# ap.add_argument('-o','--output',required=True,
#     help='Path to the output directory')
# ap.add_argument('-d','--delay',type=float,default=5.0,
#     help='Delay in seconds between frames captured')
# ap.add_argument('-dp','--display',type=int,default=0,
#     help='Boolean used to indicate if frames should be displayed')

# arg=vars(ap.parse_args())

OUTPUT_PATH='my_image/output_dir'
DELAY=2
DISPLAY=True


# initialize the output directory path and create the output directory 
output_dir=os.path.join(OUTPUT_PATH,
        datetime.now().strftime('%Y-%m-%d-%H%M'))
os.mkdir(output_dir)

# initialize the video stream 
print('[INFO] warming up camera ...')

url='http://192.168.0.116:8080/video'
vs=VideoStream(src=url,resolution=(1920,1280), framerate=30).start()
time.sleep(2.0)

# set the frame count to zero 
count=0

# signal trap to handle keyboard interrupt 
signal.signal(signal.SIGINT,signal_handler)
print('[INFO] press `ctrl + c` to exit, or "q" to quit if you have the display option on ...')



# loop over the frames from the video stream 
while True:
    # grab the frame from the video stream and resize it
    frame= vs.read()
    
    # draw the timestamp on the frame
    ts=datetime.now().strftime('%A %d %B %Y %I:%M:%S%p')
    cv.putText(frame,ts,(10,frame.shape[0]-10),cv.FONT_HERSHEY_SIMPLEX,0.35,(0,0,255),1)

    # write the current frame to output directory
    filename=f'{str(count).zfill(16)}.jpg' 
    cv.imwrite(os.path.join(output_dir,filename),frame)

    # display the frame and detect keypressess if the flag is set
    if DISPLAY:
        # show the output frame
        cv.imshow('Frame',frame)
        key= cv.waitKey(1) & 0xFF

        # if the 'q' ker was pressed, break from the loop
        if key == ord('q'):
            break
        
        # increment the count
        count+=1

        # sleep for specified number of seconds
        time.sleep(DELAY)

# do a bit for cleanup
print('[INFO] cleaning up ...')
if DISPLAY:
    cv.destroyAllWindows()
vs.stop()















[INFO] warming up camera ...
[INFO] press `ctrl + c` to exit, or "q" to quit if you have the display option on ...
[INFO] cleaning up ...


##### Processing Time Lapse Images into a Video 

In [4]:
# !pip install progressbar

Using legacy 'setup.py install' for progressbar, since package 'wheel' is not installed.
    Running setup.py install for progressbar ... done


In [7]:
from imutils.video import VideoStream
from imutils import paths
import progressbar
import signal
import time
import sys
import os

In [10]:
int('my_image/output_dir/2022-05-07-1256/0000000000000000.jpg'.split(os.path.sep)[-1][:-4])

0

In [24]:
INPUT_PATH='my_image/output_dir/2022-05-07-1256/'
OUTPUT_PATH='my_image/video_dir'
# Frames per secend
FPS=15

# function to get the frame number from the image path
def get_number(img_paths):
    return int(img_paths.split(os.path.sep)[-1][:-4])



# initialize the FourCC and video writer
fourcc=cv.VideoWriter_fourcc(*'MJPG')
writer=None

# grab the paths to the images, and initialize output file name and output path
img_paths=list(paths.list_images(INPUT_PATH))
output_file=f'{INPUT_PATH.split(os.path.sep)[2]}.avi'
output_path=os.path.join(OUTPUT_PATH,output_file)

print(f'[INFO] building {output_path} ...')


# initialize the progress bar
widgets=['Building Video : ',progressbar.Percentage(),' ',progressbar.Bar(),' ', progressbar.ETA()]
pbar=progressbar.ProgressBar(maxval=len(img_paths),widgets=widgets).start()

# loop over all stored image paths
for (i,img_path) in enumerate(sorted(img_paths,key=get_number)):
    # print('helooo')
    # load the image
    img=cv.imread(img_path)

    # initialize the video writer if needed
    if writer is None:
        (H,W) = img.shape[:2]
        writer=cv.VideoWriter(output_path,fourcc,FPS,(W,H),True)
    
    # write the image to output video
    writer.write(img)
    pbar.update(i)


# release the writer object 
print('[INFO] cleaning up ...')
pbar.finish()
writer.release()



[INFO] building my_image/video_dir/2022-05-07-1256.avi ...


[INFO] cleaning up ...


#### Bird Feeder Monitor

In [4]:
%pip install json_minify

Note: you may need to restart the kernel to use updated packages.


###### JSON-minify : 
`JSON-minify` minifies blocks of JSON-like content into valid JSON by removing
all whitespace *and* JS-style comments (single-line `//` and multi-line
`/* .. */`).

With `JSON-minify`, you can maintain developer-friendly JSON documents, but
minify them before parsing or transmitting them over-the-wire.

###### json.loads() : 
json. loads() method can be used to parse a valid JSON string and convert it into a Python Dictionary

In [12]:
confPath='Resources/src/bird feeder monitor/config/gmg.json'
json.loads(json_minify(open(confPath).read()))
# json_minify(open(confPath).read())

{'picamera': False,
 'bg_sub': 'GMG',
 'erode': {'kernel': [3, 3], 'iterations': 2},
 'dilate': {'kernel': [5, 5], 'iterations': 3},
 'min_radius': 80,
 'keyclipwriter_buffersize': 50,
 'codec': 'MJPG',
 'write_snaps': True,
 'frames_between_snaps': 30,
 'annotate': True,
 'display': True,
 'output_path': 'Resources/src/bird feeder monitor/output_gmg',
 'fps': 20}

In [1]:
from Resources.src.utils.config import Conf
from Resources.src.utils.clipwriter import KeyClipWriter
from imutils.video import VideoStream
import numpy as np
import datetime 
import imutils
import time 
import sys
import cv2 as cv
import os
 



In [6]:
# path to the JSON configuration file 
CONF_PATH='Resources/src/bird_feeder_monitor/config/gmg.json'
# path to the optional input video file 
VIDEO_PATH='Resources/src/bird_feeder_monitor/birds_10min.mp4'
# VIDEO_PATH=''

# STREAM_URL='http://192.168.0.190:8080/video'
STREAM_URL=''



In [8]:
%pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [7]:

# load our configuration settings
conf=Conf(CONF_PATH)
# conf.__getitem__('fps')

# check if we are using a camera and start video stream 
if not VIDEO_PATH:
    vs=VideoStream(src=STREAM_URL,resolution=(1920,1280), framerate=30,usePiCamera=conf['picamera']).start()
    time.sleep(3.0)
else:
    print(f'[INFO] opening video file {VIDEO_PATH}')
    vs=cv.VideoCapture(VIDEO_PATH)

# OpenCV background subtraction
OPENCV_BG_SUBTRACTIONS={
    'CNT': cv.bgsegm.createBackgroundSubtractorCNT(),
    'GMG': cv.bgsegm.createBackgroundSubtractorGMG(),
    'MOG': cv.bgsegm.createBackgroundSubtractorMOG(),
    'GSOC': cv.bgsegm.createBackgroundSubtractorGSOC(),
    'LSBP': cv.bgsegm.createBackgroundSubtractorLSBP()
}

# create our background substractor
fgbg=OPENCV_BG_SUBTRACTIONS[conf['bg_sub']]

# create erosion and dilation kernels
eKernel=np.ones(tuple(conf['erode']['kernel']),'uint8')
dKernel=np.ones(tuple(conf['dilate']['kernel']),'uint8')

# initialize key clip writer, the consecutive number of frames without 
# motion and frame since the last snapshot was written
kcw=KeyClipWriter(bufSize=conf['keyclipwriter_buffersize']) 
frames_without_motion=0
frames_since_snap=0

# begin captureing "ctrl+c" signals
# signal.signal(signal.SIGINT,signal_handler)
images=' and images ...' if conf['write_snaps'] else '...'
print(f'[INFO] detecting motion and storing videos{images}') 

# loop over the frames 
while True:
    # grab a frame from the video stream
    full_frame=vs.read()
    
    # if no frames was read, the stream has ended
    if full_frame is None:
        break
    
    # handle the frame whether the frame was read from a VideoCapture
    # or VideoStream
    full_frame=full_frame[1] if VIDEO_PATH else full_frame
    
    # increment number of frames since last snapshot was written
    frames_since_snap+=1

    # resize the frame apply the background subtractor to generate motion mask
    frame=imutils.resize(full_frame,width=500)
    mask=fgbg.apply(frame)
    
    # perform erosions and dilations to eliminate noise and fill gaps
    mask=cv.erode(mask,eKernel,iterations=conf['erode']['iterations'])
    mask=cv.dilate(mask,dKernel,iterations=conf['dilate']['iterations'])

    # find contours in the mask and reset the motion status
    cnts=cv.findContours(mask.copy(),cv.RETR_EXTERNAL,
                         cv.CHAIN_APPROX_SIMPLE)
    cnts=imutils.grab_contours(cnts)
    motion_this_frame=False
    
    # loop over the contours
    for c in cnts:
        # It is a circle which completely covers the object with minimum area
        ((x,y),radius)=cv.minEnclosingCircle(c)
        (rx,ry,rw,rh)=cv.boundingRect(c)
        
        # convert floating point values to integers
        (x,y,radius)=[int(v) for v in (x,y,radius)]        
        
        # only process motion contours above the specified size
        if radius < conf['min_radius']:
            continue
        # grab the current timestamp 
        timestamp=datetime.datetime.now()
        timestring=timestamp.strftime('%Y%m%d-%H%M%S')
        
        # set our motion flag to indicate we have found motion and
        # reset the motion counter
        motion_this_frame=True
        frames_without_motion=0
        
        # check if we need to annotate the frame for display
        if conf['annotate']:
            cv.circle(frame,(x,y),radius,(0,0,255),2)
            cv.rectangle(frame,(rx,ry),(rx+rw,ry+rh),(0,255,0),2)
        
        # frame to disk
        write_frame=frames_since_snap >= conf['frames_between_snaps']
        
        # check to see if should write the frame to disk
        if conf['write_snaps'] and write_frame:
            # construct the path to output photo and save it
            snap_path=os.path.sep.join([conf['output_path'],timestring])
            cv.imwrite(snap_path+'.jpg',full_frame)
            
            # reset the counter between snapshots
            frames_since_snap=0
        
        # start recording if we aren't already
        if not kcw.recording:
            # construct the path to the video file
            video_path=os.path.sep.join([conf['output_path'],timestring])

            # instantiate the video codec object and start the key clip writer
            fourcc=cv.VideoWriter_fourcc(*conf['codec'])
            kcw.start('{}.avi'.format(video_path),fourcc,conf['fps']) 
        
        # check if no motion was detected in this frame and then increment 
        # the number of consecutive frames without motion
        if not motion_this_frame:
            frames_without_motion+=1
            
        # update the key clip buffer
        kcw.update(frame)
    
        # check to see if the number of frames without motion is above our defined threshold
        no_motion=frames_without_motion >= conf['keyclipwriter_buffersize']
        
        # stop recording if there is no motion
        if kcw.recording and no_motion:
            kcw.finish()
        
        # check to see if we'er displaying the frame to our screen
        if conf['display']:
            # display the frame and grab keypresses
            cv.imshow('Frames',frame) 
            key=cv.waitKey(1) & 0xFF
            
            # if the 'q' ker was pressed, break from the loop
            if key==ord('q'):
                break
            
print('[INFO] cleaning up ...')
# check if we'er recording and stop recording 
if kcw.recording:
    kcw.finish()

# stop the video stream
vs.stop if not VIDEO_PATH else vs.release()        
                   


[INFO] opening video file Resources/src/bird_feeder_monitor/birds_10min.mp4
[INFO] detecting motion and storing videos and images ...


KeyboardInterrupt: 

#### Sending Notification From RPI